# PostgreSQL

In [1]:
import sqlalchemy as db
import zeit
import pandas as pd
import configparser

In [8]:
c = configparser.ConfigParser()
c.read("config.ini")
zeit_key = c.get("Zeit", "key")
uri = c.get("Postgres", "URI")

#set the API, authenticate and check status
api = zeit.API()
api.set_token(zeit_key)
api.get_status()

'everything ok'

In [3]:
engine = db.create_engine("postgresql://code@localhost/test") #local
#engine = db.create_engine(uri) #on herokus
con = engine.connect()
metadata = db.MetaData()

### Retrieving Values

In [4]:
table = db.Table("keyword", metadata, autoload=True, autoload_with=engine)

In [16]:
word = "Merkel"
query = db.select([table])#.where(table.columns.uri == f"http://api.zeit.de/keyword/{word}")
ResultProxy = con.execute(query)
result = ResultProxy.fetchmany(5)
try:
    df = pd.DataFrame(result, columns = result[0].keys())
except Exception:
    print("No Results")

In [17]:
df

,uri,lexical,results,type,json,matches,score
0,http://api.zeit.de/keyword/mecklenburg-vorpommern,Mecklenburg-Vorpommern,1324,location,"{""-750236879496"":3,""-718700879496"":1,""-6555424...",{'subtitle': 'Alfred Sauter war einer der einf...,68
1,http://api.zeit.de/keyword/krankenkassen,Krankenkassen,1,organization,"{""1269449521497"":1}",{'subtitle': 'Wer nach einem Burn-out wieder a...,44
2,http://api.zeit.de/keyword/gazastreifen,gazastreifen,183,subject,"{""1206377522496"":2,""1269449522496"":1,""13326079...",{'subtitle': 'Erstmals seit Jahren ist der Gre...,53
3,http://api.zeit.de/keyword/angela-merkel,Angela Merkel,9400,person,"{""638307854978"":5,""669843854978"":9,""7014662549...","{'subtitle': 'Wer entscheidet, wenn man es sel...",93
4,http://api.zeit.de/keyword/armin-laschet,Armin Laschet,473,person,"{""859232655982"":1,""1016999055982"":1,""108015745...","{'subtitle': 'Das ist neu, nicht nur für die C...",9


### Inserting Values

In [17]:
table.columns.keys()

['uri', 'lexical', 'results', 'type', 'json', 'matches', 'score']

In [35]:
r = api.get_keyword("armin-laschet", facet_time="1year")

In [36]:
query = db.insert(table).values(
    uri = r.uri, lexical = r.name, results = r.found, 
    type = r.type, json = r.get_facets().to_json(),
    matches = r.get_raw()["matches"][0], score = r.score
)
ResultProxy = con.execute(query)

### Deleting Values

In [32]:
delete = table.delete().where(table.columns.lexical == 'Laschet, Armin')
ResultProxy = con.execute(delete)

## Workflow for inserting

In [12]:
api.search_for("spahn", "keyword")

 Search for 'spahn': 1 results, limit: 10, matches : 
 
Jens Spahn: http://api.zeit.de/keyword/jens-spahn

In [18]:
word_list = ["friedrich-merz", "jens-spahn"]
for w in word_list:
    #call the api
    r = api.get_keyword(w, facet_time="1year", limit = 1)

    try:
        matches = r.get_raw()["matches"][0]
    except Exception:
        matches = ''

    #construct the insert query
    query = db.insert(table).values(
        uri = r.uri, lexical = r.name, results = r.found, 
        type = r.type, json = r.get_facets().to_json(),
        matches = matches, score = r.score,
        )
    #execute it
    ResultProxy = con.execute(query)

# Get the Data in Dash

In [ ]:
word = "gazastreifen"
query = db.select([table]).where(table.columns.uri == f"http://api.zeit.de/keyword/{word}")
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()

In [ ]:
#better querys
query = db.select([table]).filter(db.or_( #SELECT * FROM table WHERE 
        table.columns.uri.like(f"%meck%"), #uri LIKE '%word%' OR
        table.columns.lexical.like(f"%meck%"), #lexical LIKE '%word%' OR
        )
        ).order_by(db.desc(table.columns.results)).limit(5) #ORDER BY results DESC LIMIT 5
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()